In [ ]:
from copy import deepcopy
import hubbard as hbb
import numpy as np
import scipy.sparse as sp
from qiskit import AncillaRegister, ClassicalRegister, QuantumCircuit
import qmatchatea as qtea
from qmatchatea.py_emulator import QcMps
from qmatchatea.preprocessing import _preprocess_qk

from hubbard.hamiltonian_terms import hopping_hamiltonian
from hubbard.utils import _lattice_state, _printable_site_str
import os

import matplotlib.pyplot as plt

In [ ]:
# Vertexes definition
shape = (4, 2)
vertexes = [(ii, jj) for jj in range(shape[1]) for ii in range(shape[0])]
sites = [f"q({ii}, {jj})" for jj in range(shape[1]) for ii in range(shape[0])]
site_ordering = []
idx = 0
for ii, jj in vertexes:
    if (ii+jj)%2==0:
        site_ordering += [idx, idx+1]
    else:
        site_ordering += [idx+1, idx]
    idx += 2
# Plaquettes definition
plaquettes = [(ii, jj) for jj in range(shape[1]-1) for ii in range(shape[0]-1) ]
# Number of link qubits
num_links = shape[0]*(shape[1]-1) + shape[1]*(shape[0]-1)
num_qubs = 2*len(vertexes) + num_links + 1
qancilla = AncillaRegister(1, 'a0')
cancillas = [ClassicalRegister(1, f'ca{ii}') for ii in range(len(plaquettes)+len(vertexes)+1)]

# ============= Initialize Hubbard circuit =============
regs, qc = hbb.hubbard_circuit(shape, qancilla, cancillas )

In [ ]:
# Define the Hamiltonian
onsite_const = 1
hopping_const = 0.1
total_hamiltonian = {}
onsite_hamiltonian = hbb.onsite_hamiltonian(qc, regs, shape, onsite_const)
#hopping_hamiltonian = hbb.hopping_hamiltonian(qc, regs, shape, hopping_const)
#total_hamiltonian.update(onsite_hamiltonian)
#hopping_hamiltonian = hbb.generate_global_hopping(qc, regs, "lh0", "u", coupling=hopping_const)
#total_hamiltonian.update(hopping_hamiltonian)
hopping_hamiltonian = hbb.generate_global_hopping(qc, regs, "lh2", "d", coupling=hopping_const)
total_hamiltonian.update(hopping_hamiltonian)

In [ ]:
for term, coef in total_hamiltonian.items():
    lattice = _lattice_state(qc, term, regs, shape)
    lattice_string = f"{coef}\n"
    for y_string_sites in lattice[::-1]:
        lattice_string += _printable_site_str(y_string_sites)
    
    print(lattice_string)
    print("")

In [ ]:
def simple_visualizer(shape, state):
    state_str = []
    current_str = "│"
    vertical = " "*2+"│     "*shape[0]
    top = "┌─┴─┐ "*shape[0]
    bottom = "└─┬─┘ "*shape[0]
    for jj in range(shape[1]):
        state_str.append(bottom + "\n")
        for ii in range(shape[0]):
            if ii>0:
                current_str += "┤"
            idx = 2*(ii+ jj*shape[0])
            current_str += state[idx]+","+state[idx+1]+"├─"
        state_str.append(current_str[:-1]+"\n")
        state_str.append(top + "\n")
        if jj < shape[1]-1:
            state_str.append(vertical+"\n")
        current_str = "│"
    
    return "".join(state_str[::-1])

In [ ]:
shape = (4, 2)

save_dir = f"data/exact/{shape[0]}x{shape[1]}"
hamiltonian = np.loadtxt(os.path.join(save_dir, "symmetric_hamiltonian.txt"), dtype=complex)
hamiltonian_jw = np.loadtxt(os.path.join(save_dir, "symmetric_hamiltonian_jw.txt"), dtype=complex)

all_states = hbb.all_possible_matter_states(shape, 4, 4)
all_state_strings = ["".join(list(state)) for state in all_states.astype(str) ]

In [ ]:
len(all_state_strings)

In [ ]:
# Alto dx, alto sx, basso dx, basso sx
for ii, state in enumerate(all_state_strings):
    print(ii)
    print( simple_visualizer(shape, state) )

In [ ]:
#path = [0, 3, 4, 2, 5, 3, 1, 2, 0]
#path = [3, 4, 2, 5, 3]#, 4, 2, 5, 3]
#path = [0, 1, 9, 4, 0]
path = [0, 1, 4, 9, 49, 36, 25, 16, 0]
path = [0, 1, 4, 36, 25, 16, 0]
path = [0, 1, 25, 16, 0]

In [ ]:
path = path[::-1]
terms = []
for ii in range(len(path)-1):
    terms.append(
        hamiltonian[path[ii], path[ii+1]]
    )
    #print(f"from {path[ii]} to {path[ii+1]}")

prod = 1
for term in terms:
    prod *= term/np.abs(term)

print(prod)

In [ ]:
terms = []
for ii in range(len(path)-1):
    terms.append(
        hamiltonian_jw[path[ii], path[ii+1]]
    )

prod = 1
for term in terms:
    prod *= term/np.abs(term)

print(prod)

In [ ]:
idx = 25
print( np.nonzero(hamiltonian[idx, :]))
print( np.nonzero(hamiltonian_jw[idx, :]))

In [ ]:
hamiltonian_jw[25, 16]

In [ ]:
(hamiltonian[25, 16], 2)

In [ ]:
UU = np.diag(
    [1j, 1j, 1j, 1j, 1j, 1j]
)

In [ ]:
np.conj(UU.T)@hamiltonian@UU/0.1